In [ ]:
!pip install serpapi

In [1]:
import numpy as np
import gensim
import nltk
from gensim.models import Word2Vec
from sentence_transformers.util import cos_sim
from sentence_transformers import SentenceTransformer, util
import tensorflow as tf
import tensorflow_hub as hub
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle

In [2]:
word_model = Word2Vec.load("word_model.model")

# create an instance of ELMo
elmo = hub.load("https://tfhub.dev/google/elmo/2")
with open('sentences.data', 'rb') as fp:
        sentences = pickle.load(fp)
with open('sents_inELMo.data', 'rb') as fp:
        filtered = pickle.load(fp)
with open('elmo_vect.data', 'rb') as fp:
        elmo_vect= pickle.load(fp)

sents_model = SentenceTransformer('all-MiniLM-L6-v2')
sents_embeddings = np.load("sents_embedding.npy")


user enter query
show related keywords (repeat = loops= iteration) + first search
result classify, show in multiple tabs (All, definition, tutorial, etc.)


In [114]:
user_query = input("Enter your query: ")
buffer_query = ""

# identify non english words - take it as website name
website_dict = ["stackoverflow" , "quora" , "geeksforgeeks", "reddit" , "tutorialspoint",
               "codecademy" , "w3schools"]
#named entity reconigtion -try

# POS tagging
query_tokens = user_query.split()
tokens_tag = pos_tag(query_tokens)
print(tokens_tag)

for word in query_tokens:
    if word.lower() in website_dict:
        buffer_query += ":" + word.lower() + " "
        index = query_tokens.index(word)
        del tokens_tag[index]
        del query_tokens[index]
        if tokens_tag[index-1][1] == "IN" :
            del tokens_tag[index-1]
            del query_tokens[index-1]
        
processed_query = " ".join([q for q in query_tokens])

print("processed_query : " + processed_query)
print("buffer_query : " + buffer_query)

#spelling check + suggestion for like google recommend

Enter your query: for loop and while loop
[('for', 'IN'), ('loop', 'NN'), ('and', 'CC'), ('while', 'IN'), ('loop', 'NN')]
processed_query : for loop and while loop
buffer_query : 


In [115]:
# query analysis
stop_words = set(stopwords.words('english'))
exc =["if","for","do","while","same","all","where","how","when","what","which","who"]
for w in exc:
    stop_words.remove(w)

print("\nMost similar words : \n")
# find similar words and sentences
for t in tokens_tag:
    if t[0] not in stop_words and t[1] != "IN":
        # find similar words
        print(t[0] + "-->")
        try:
            similar_words = ", ".join([w[0] for w in word_model.wv.most_similar(t[0],topn=3)])
            print(similar_words)
        except:
            print("")
        
#find possible sentences using keyword
#extract keyword from query
import yake
kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 2
deduplication_threshold = 0.9
numOfKeywords = 1
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            top=numOfKeywords, features=None)
search_string = custom_kw_extractor.extract_keywords(processed_query)

print("\nPossible sentences : \n")
#find possible sentences
elmo_embeddings2 = elmo.signatures["default"](tf.constant([search_string[0][0]]))["default"]
search_vect = elmo_embeddings2.numpy()
search_vect = search_vect.reshape(1, -1)

cosine_similarities = np.dot(search_vect, elmo_vect.T) / (
np.linalg.norm(search_vect) * np.linalg.norm(elmo_vect, axis=1))

output = ""
for i in cosine_similarities.argsort()[0][-3:]:
    output += filtered[i] + "\n"
print(output)

print("\nMost related sentences : \n")
#find similar sentences for query
target_query = [processed_query]
query_embedding = sents_model.encode(target_query)
sim = np.zeros((len(target_query), len(sentences)))

for i in range(len(target_query)):
    for j in range(len(sentences)):
        sim[i,j] = cos_sim(query_embedding[i], sents_embeddings[j])

most_related = sorted(zip(sim[0], sentences), reverse=True)[:3]
related_sents=""
for i in most_related:
    related_sents += i[1] + "\n"
print(related_sents)


Most similar words : 

loop-->
do-while, statement, continue
loop-->
do-while, statement, continue

Possible sentences : 

loops while and fr
exit from loop
do-while loops


Most related sentences : 

while loops
the for statement is a loop a generalization of the while
the while do and for statements support iteration looping



In [116]:
# suggestion + proceed search
search_query = processed_query + buffer_query

In [117]:
# metasearch module

from serpapi import GoogleSearch
import json

link = {"Title":"","Url":"","Summary":"","Keywords":""};
links = []
start = 0
end = 100
page_size = 25

params = {
  "engine": "duckduckgo",
  "q": search_query,
  "api_key": "724fa01b8ad06ed4dbaf6cd1d67fa94b3e6fdc8cb9952cd56e19818eeb9e6bec",
  "start": start,
  "end": end,
  "num": page_size
}

search = GoogleSearch(params)
results = search.get_dict()
# create a python generator using parameter
pages = search.pagination(start, end, page_size)

organic_results = results["organic_results"]

# fetch one search result per iteration 
# using a basic python for loop 
# which invokes python iterator under the hood
for page in pages:
    for search_result in page["organic_results"]:
        link["Title"] = search_result['title']
        link["Url"] = search_result['link']
        links.append(link.copy())


In [118]:
for link in links:
    print(link)

{'Title': 'loops - When to use "while" or "for" in Python - Stack Overflow', 'Url': 'https://stackoverflow.com/questions/920645/when-to-use-while-or-for-in-python', 'Summary': '', 'Keywords': ''}
{'Title': 'How to Pick Between a For Loop and While Loop | Built In', 'Url': 'https://builtin.com/software-engineering-perspectives/for-loop-vs-while-loop', 'Summary': '', 'Keywords': ''}
{'Title': 'Iteration statements -for, foreach, do, and while | Microsoft Learn', 'Url': 'https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/statements/iteration-statements', 'Summary': '', 'Keywords': ''}
{'Title': 'Difference Between for and while loop - Online Tutorials Library', 'Url': 'https://www.tutorialspoint.com/difference-between-for-and-while-loop', 'Summary': '', 'Keywords': ''}
{'Title': 'Difference between for loop and while loop in Python', 'Url': 'https://www.geeksforgeeks.org/difference-between-for-loop-and-while-loop-in-python/', 'Summary': '', 'Keywords': ''}
{'Title': 'Loops

In [119]:
import urllib.request as urllib2
import urllib.error
import socket
from bs4 import BeautifulSoup

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

import lxml
from lxml.html import document_fromstring
from lxml.html.clean import Cleaner
import yake

In [120]:
# get keywords for each results
for link in links:

#html text preprocessing
    if "pdf" not in link["Url"]:
        url = link["Url"]
    else:
        continue
    
    req = urllib2.Request(url, headers = {'User-Agent': 'Mozilla/5.0 Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'})
    
    try:
        html = urllib2.urlopen(req, timeout=5).read()
    except urllib.error.HTTPError as e:
        print('HTTPError: {}'.format(e.code))
    except urllib.error.URLError as e:
        print('URLError: {}'.format(e.reason))
    except socket.timeout as e:
        print('Timeout Error')
    except:
        print('Unknown Error')
    else:
        cleaner = Cleaner()
        cleaner.javascript = True # This is True because we want to activate the javascript filter
        cleaner.style = True      # This is True because we want to activate the styles & stylesheet filter
        doc = document_fromstring(html)
        cleaned_html = lxml.html.tostring(cleaner.clean_html(doc))

        raw = BeautifulSoup(cleaned_html, 'html.parser').get_text()
        
        parser = PlaintextParser.from_string(raw,Tokenizer("english"))
        
        summarizer_lex = LexRankSummarizer()
        # Summarize using sumy LexRank
        summary = summarizer_lex(parser.document, 4)         
        link["Summary"] = summary

Timeout Error
Timeout Error
HTTPError: 502
HTTPError: 403
Timeout Error


In [121]:
for link in links:
    print(link["Summary"])

(<Sentence: loops - When to use "while" or "for" in Python - Stack Overflow>, <Sentence: When I should use a while loop or a for loop in Python?>, <Sentence: A for loop will iterate through a list (a range being a list of numbers).>, <Sentence: Provide details and share your research!But avoid â€¦Asking for help, clarification, or responding to other answers.Making statements based on opinion; back them up with references or personal experience.To learn more, see our tips on writing great answers.>)
(<Sentence: What Is a For Loop?A for loop is more structured than the while loop.>, <Sentence: The keyword for is used, followed by three statements:Initialization: Executed before the loop begins.Expression: Evaluated before each iteration, exits the loop when false.Increment: Executed at the end of each iteration.for(count=1; count < 10; count++) { console.log(count); }The increment does not have to be ++, but you’ll see that the most often.So, in summary, the while loop has a looser synt

In [122]:
lemmatizer = WordNetLemmatizer()

for link in links:
#html text preprocessing
    if "pdf" not in link["Url"]:
        url = link["Url"]
    else:
        continue
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 3
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                                dedupLim=deduplication_threshold, 
                                                top=numOfKeywords, features=None)
    
    summary=""
    for sentence in link["Summary"]:
        summary+=str(sentence) + " "
    extracted_keywords = custom_kw_extractor.extract_keywords(summary)

    keywords = []
    for keyword in extracted_keywords:
        keywords.append(lemmatizer.lemmatize(keyword[0].lower()))
    link["Keywords"] = keywords
    
for link in links:
    print(link['Url'] + " " + str(link['Keywords']))

https://stackoverflow.com/questions/920645/when-to-use-while-or-for-in-python ['python', 'stack', 'overflow']
https://builtin.com/software-engineering-perspectives/for-loop-vs-while-loop ['loop', 'count', 'executed']
https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/statements/iteration-statements ['statement', 'foreach', 'initializer']
https://www.tutorialspoint.com/difference-between-for-and-while-loop ['loop', 'java', 'difference']
https://www.geeksforgeeks.org/difference-between-for-loop-and-while-loop-in-python/ []
https://javascript.info/while-for ['loop', 'truthy', 'form']
https://stackoverflow.com/questions/2950931/for-vs-while-in-c-programming ['overflow', 'team', 'loop']
https://www.khanacademy.org/computing/computer-programming/programming/looping/e/review-loops []
http://www.differencebetween.net/technology/difference-between-for-and-while-loop/ ['difference', 'loop', 'application']
https://www.pythonforbeginners.com/basics/for-loop-vs-while-loop-in-python 

In [123]:
for link in links:
    print(link['Url'] + " " + str(link['Keywords']))
    
groups_name = []
groups_total = []
for link in links:
    for keyword in link["Keywords"]:
        if keyword not in groups_name:
            groups_name.append(keyword)
            groups_total.append(1)
        else:
            groups_total[groups_name.index(keyword)] += 1

for i in range(len(groups_name)):
    print(groups_name[i] + " " + str(groups_total[i]))

https://stackoverflow.com/questions/920645/when-to-use-while-or-for-in-python ['python', 'stack', 'overflow']
https://builtin.com/software-engineering-perspectives/for-loop-vs-while-loop ['loop', 'count', 'executed']
https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/statements/iteration-statements ['statement', 'foreach', 'initializer']
https://www.tutorialspoint.com/difference-between-for-and-while-loop ['loop', 'java', 'difference']
https://www.geeksforgeeks.org/difference-between-for-loop-and-while-loop-in-python/ []
https://javascript.info/while-for ['loop', 'truthy', 'form']
https://stackoverflow.com/questions/2950931/for-vs-while-in-c-programming ['overflow', 'team', 'loop']
https://www.khanacademy.org/computing/computer-programming/programming/looping/e/review-loops []
http://www.differencebetween.net/technology/difference-between-for-and-while-loop/ ['difference', 'loop', 'application']
https://www.pythonforbeginners.com/basics/for-loop-vs-while-loop-in-python 

In [124]:
import pickle
with open('links_completed.data', 'wb') as fp:
        pickle.dump(links, fp)

In [125]:
#lemmatize keywords
# bert api classification

#pdf need to exclude or utf8 error decode
#video can't generate keywords

In [126]:
import pickle
# Open the file for reading
with open('links_completed.data', 'rb') as fp:
        links = pickle.load(fp)

In [127]:
# Group most 10 frequent
#link = {"Title":"","Url":"","Summary":"","Keywords":""};
group = {"Group_Name":"", "Group_Total":"", "Group_Links":[] , "Group_Scores":[]}
groups = []

groups_name = []
groups_total = []
for link in links:
    for keyword in link["Keywords"]:
        if keyword not in groups_name:
            groups_name.append(keyword)
            groups_total.append(1)
        else:
            groups_total[groups_name.index(keyword)] += 1
    
topten = sorted(zip(groups_total, groups_name), reverse=True)[:10]
print(topten)

for g in topten:
    group["Group_Total"] = g[0]
    group["Group_Name"] = g[1]
    for link in links:
        for keyword in link["Keywords"]:
            if keyword == g[1]:
                group["Group_Links"].append(link["Url"])
                break
    groups.append(group.copy())
    
print(groups[0])

[(61, 'loop'), (10, 'python'), (9, 'tutorial'), (8, 'condition'), (7, 'number'), (7, 'community'), (6, 'print'), (6, 'overflow'), (6, 'difference'), (5, 'statement')]
{'Group_Name': 'loop', 'Group_Total': 61, 'Group_Links': ['https://builtin.com/software-engineering-perspectives/for-loop-vs-while-loop', 'https://www.tutorialspoint.com/difference-between-for-and-while-loop', 'https://javascript.info/while-for', 'https://stackoverflow.com/questions/2950931/for-vs-while-in-c-programming', 'http://www.differencebetween.net/technology/difference-between-for-and-while-loop/', 'https://www.pythonforbeginners.com/basics/for-loop-vs-while-loop-in-python', 'https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 'https://www.makeuseof.com/for-while-do-while-java/', 'https://realpython.com/python-while-loop/', 'https://techdifferences.com/differenece-between-for-and-while-loop.html', 'https://askanydifference.com/difference-between-for-loop-and-while-loop/', 'https://www.veda

In [128]:
import yake

# ranking by weight
def cal_similar (processed_query, group_name, summary, link_keywords):
    #extract keyword from user query
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 1
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                                dedupLim=deduplication_threshold, 
                                                top=numOfKeywords, features=None)
    user_keywords = custom_kw_extractor.extract_keywords(processed_query)
    keyword = user_keywords[0][0]
    
    #calculate word similarity score
    word_score = 0
    for link_keyword in link_keywords:
        #word_score += model.wv.similarity(string,string)
        try:
            word_score += word_model.wv.similarity(keyword,link_keyword)
        except:
            word_score += 0
        try:
            word_score += word_model.wv.similarity(group_name,link_keyword)
        except:
            word_score += 0
    
    #calculate sentence similarity score
    sents_score = 0
    #sents_model.encode(['string'])
    query_embedding1 = sents_model.encode([processed_query])
    for sentence in summary:
        query_embedding2 = sents_model.encode([str(sentence)])
        sents_score += cos_sim(query_embedding1, query_embedding2)
        
    #calulate total similarity score
    total_score = 60/100*word_score + 40/100*sents_score
    return float(total_score)

for group in groups:
    temp_links = []
    temp_score = []
    temp_links = group["Group_Links"].copy()
    for temp_link in temp_links:
        for link in links:
            if temp_link == link["Url"]:
                summary = link["Summary"]
                link_keywords = link["Keywords"]
        sim_score =  cal_similar (processed_query, group["Group_Name"], summary, link_keywords)
        temp_score.append(sim_score)
    temp = sorted(zip(temp_links, temp_score), reverse=True)
    print(temp)
    temp_links = [t[0] for t in temp]
    temp_score = [t[1] for t in temp]
    group["Group_Links"] = temp_links
    group["Group_Scores"] = temp_score

[('https://www.w3schools.com/python/python_while_loops.asp', 0.6513859033584595), ('https://www.w3schools.com/python/python_while_loops.asp', 0.6513859033584595), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.6432759761810303), ('https://www.w3schools.com/python/py

[('https://www.w3schools.com/python/python_while_loops.asp', 1.1384763717651367), ('https://www.w3schools.com/python/python_while_loops.asp', 1.1384763717651367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.3905539512634277), ('https://www.w3schools.com/python/py

[('https://www.w3schools.com/python/python_while_loops.asp', 1.5006099939346313), ('https://www.w3schools.com/python/python_while_loops.asp', 1.5006099939346313), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5554052591323853), ('https://www.w3schools.com/python/py

[('https://www.w3schools.com/python/python_while_loops.asp', 0.5870313048362732), ('https://www.w3schools.com/python/python_while_loops.asp', 0.5870313048362732), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.5474950075149536), ('https://www.w3schools.com/python/py

[('https://www.w3schools.com/python/python_while_loops.asp', 0.4964395761489868), ('https://www.w3schools.com/python/python_while_loops.asp', 0.4964395761489868), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.297118902206421), ('https://www.w3schools.com/python/python_for

[('https://www.w3schools.com/python/python_while_loops.asp', 0.38912534713745117), ('https://www.w3schools.com/python/python_while_loops.asp', 0.38912534713745117), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.0321531295776367), ('https://www.w3schools.com/python/

[('https://www.w3schools.com/python/python_while_loops.asp', 0.3806638717651367), ('https://www.w3schools.com/python/python_while_loops.asp', 0.3806638717651367), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.4682875871658325), ('https://www.w3schools.com/python/py

[('https://www.w3schools.com/python/python_while_loops.asp', 0.7867315411567688), ('https://www.w3schools.com/python/python_while_loops.asp', 0.7867315411567688), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.391459345817566), ('https://www.w3schools.com/python/python_for

[('https://www.w3schools.com/python/python_while_loops.asp', 0.8010932803153992), ('https://www.w3schools.com/python/python_while_loops.asp', 0.8010932803153992), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.122727632522583), ('https://www.w3schools.com/python/python_for

[('https://www.w3schools.com/python/python_while_loops.asp', 0.36224377155303955), ('https://www.w3schools.com/python/python_while_loops.asp', 0.36224377155303955), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai', 1.405849575996399), ('https://www.w3schools.com/python/python_f

In [129]:
i = 0 #choose which group to view
chosen_group = groups[i]
print(chosen_group["Group_Name"] + ": ")
for group_ele in chosen_group["Group_Links"]:
    for link in links:
        if group_ele == link["Url"]:
            print(link["Title"] + " " + link ["Url"])

loop: 
Python While Loops - W3Schools https://www.w3schools.com/python/python_while_loops.asp
Python While Loops - W3Schools https://www.w3schools.com/python/python_while_loops.asp
Python While Loops - W3Schools https://www.w3schools.com/python/python_while_loops.asp
Python While Loops - W3Schools https://www.w3schools.com/python/python_while_loops.asp
Python For Loops - W3Schools https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai
Python For Loops - W3Schools https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai
Python For Loops - W3Schools https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai
Python For Loops - W3Schools https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai
Python For Loops - W3Schools https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai
Python For Loops - W3Schools https://www.w3schools.com/python/python_for_loops.asp?ref=dailydispatch.ai
Python For Loops - W3